In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

from tqdm import tqdm
from mobilenetv3 import *

In [2]:
# small
# net_small = mobilenetv3(mode='small')
# state_dict = torch.load('mobilenetv3_small_67.4.pth.tar')
# net_small.load_state_dict(state_dict)

# large
# net_large = mobilenetv3(mode='large')
#net_large = mobilenetv3(mode='large')

#x = torch.randn(2,3,224,224)
#y = net_large(x)
#print(y.size())
#print(y)
import os 
os.system("rm ./weights/*")

256

In [3]:
model = mobilenetv3(mode='large', n_class=2)

In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

input_size = 224
batch_size = 16
n_worker = 4

train_dir = './traindir'
train_datasets = datasets.ImageFolder(
        train_dir, transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
train_dataloader = torch.utils.data.DataLoader(
    train_datasets,
    batch_size=batch_size, shuffle=True,
    num_workers=n_worker, pin_memory=True)

val_dir = './valdir'
val_datasets = datasets.ImageFolder(
        val_dir, transforms.Compose([
        transforms.Resize(int(input_size/0.875)),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        normalize,
    ]))

val_dataloader = torch.utils.data.DataLoader(
    val_datasets,
    batch_size=batch_size, shuffle=False,
    num_workers=n_worker, pin_memory=True)

In [6]:
total_epoch = 1000
learning_rate = 0.0005
save_every = 10
log_every = 10

# --------------------训练过程---------------------------------
if torch.cuda.is_available():
    model.cuda()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, 50, 0.1)
loss_func = nn.CrossEntropyLoss()

Loss_list = []
Accuracy_list = []

for epoch in range(total_epoch):
    print(' ====== ====== ====== epoch {}'.format(epoch))
    # training-----------------------------
    train_loss = 0.
    train_acc = 0.
    model.train()
    for batch_x, batch_y in (train_dataloader):  #tqdm
        batch_x, batch_y = Variable(batch_x).cuda(), Variable(batch_y).cuda()
        out = model(batch_x)
        loss = loss_func(out, batch_y)
        train_loss += loss.item()  #loss.data[0]
        pred = torch.max(out, 1)[1]
        train_correct = (pred == batch_y).sum()
        train_acc += train_correct.item()  #data[0]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #print('Train Loss: {:.6f}, Acc: {:.6f}'.format(train_loss / (len(
        #train_datasets)), train_acc / (len(train_datasets))))

    # evaluation--------------------------------
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for batch_x, batch_y in (val_dataloader):  #tqdm
        with torch.no_grad():
            batch_x, batch_y = Variable(batch_x).cuda(), Variable(batch_y).cuda()
            out = model(batch_x)
            loss = loss_func(out, batch_y)
            eval_loss += loss.item()  #.data[0]
            pred = torch.max(out, 1)[1]
            num_correct = (pred == batch_y).sum()
            eval_acc += num_correct.item()  #.data[0]
    #print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
        #val_datasets)), eval_acc / (len(val_datasets))))

    Loss_list.append(eval_loss / (len(val_datasets)))
    Accuracy_list.append(100 * eval_acc / (len(val_datasets)))

    if epoch % save_every == 0:
        print('Save model at epoch %d'%epoch)
        # save model
        torch.save(model.state_dict(), 'weights/mobilenetv3_%d.pkl'%epoch)

    if epoch % log_every == 0:
        print('epoch {}'.format(epoch))
        #print('epoch {}, Train Loss: {:.6f}'.format((epoch), train_loss / (len(trainDataset))))
        print('Train Loss: {:.6f}, Acc: {:.6f}'.format(train_loss / (len(
                train_datasets)), train_acc / (len(train_datasets))))
        print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
                val_datasets)), eval_acc / (len(val_datasets))))

    scheduler.step()

 ====== ====== ====== epoch 0
Save model at epoch 0
epoch 0
Train Loss: 0.028018, Acc: 0.745000
Test Loss: 0.056455, Acc: 0.500000
 ====== ====== ====== epoch 1
 ====== ====== ====== epoch 2
 ====== ====== ====== epoch 3
 ====== ====== ====== epoch 4
 ====== ====== ====== epoch 5
 ====== ====== ====== epoch 6
 ====== ====== ====== epoch 7
 ====== ====== ====== epoch 8
 ====== ====== ====== epoch 9
 ====== ====== ====== epoch 10
Save model at epoch 10
epoch 10
Train Loss: 0.002239, Acc: 0.990000
Test Loss: 0.147718, Acc: 0.950000
 ====== ====== ====== epoch 11
 ====== ====== ====== epoch 12
 ====== ====== ====== epoch 13
 ====== ====== ====== epoch 14
 ====== ====== ====== epoch 15
 ====== ====== ====== epoch 16
 ====== ====== ====== epoch 17
 ====== ====== ====== epoch 18
 ====== ====== ====== epoch 19
 ====== ====== ====== epoch 20
Save model at epoch 20
epoch 20
Train Loss: 0.000363, Acc: 1.000000
Test Loss: 0.000259, Acc: 1.000000
 ====== ====== ====== epoch 21
 ====== ====== ======

KeyboardInterrupt: 

In [5]:
total_epoch = 100
learning_rate = 0.001
save_every = 10
log_every = 10

if torch.cuda.is_available():
    model.cuda()
    
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
criterion = nn.CrossEntropyLoss()
# 训练部分
for epoch in range(total_epoch):
    print(' ====== ====== ====== epoch {}'.format(epoch))
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()

        optimizer.zero_grad()        # 梯度置零，因为反向传播过程中梯度会累加上一次循环的梯度

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()                   # 反向传播后参数更新 
        running_loss += loss.item()       # loss累加
    print("    train loss: {}".format(epoch, running_loss))
    
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for batch_x, batch_y in (val_dataloader):  #tqdm
        with torch.no_grad():
            batch_x, batch_y = Variable(batch_x).cuda(), Variable(batch_y).cuda()
            out = model(batch_x)
            loss = loss_func(out, batch_y)
            eval_loss += loss.item()  #.data[0]
            pred = torch.max(out, 1)[1]
            num_correct = (pred == batch_y).sum()
            eval_acc += num_correct.item()

epoch: 0, loss: 7.879641532897949
epoch: 1, loss: 6.304314404726028
epoch: 2, loss: 5.14381268620491
epoch: 3, loss: 4.2963544726371765
epoch: 4, loss: 2.775220647454262
epoch: 5, loss: 1.8251195549964905
epoch: 6, loss: 1.2622105292975903
epoch: 7, loss: 1.2611360363662243
epoch: 8, loss: 0.7494088746607304
epoch: 9, loss: 0.6811636500060558
